In [4]:
pip install twython

  Using cached https://files.pythonhosted.org/packages/a3/12/b92740d845ab62ea4edf04d2f4164d82532b5a0b03836d4d4e71c6f3d379/requests_oauthlib-1.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [16]:
from twython import Twython

CONSUMER_KEY = 'Uj4evSaoPD0BeECqnW4QNJHQ5'
CONSUMER_SECRET = 'rS3rU1KQpBEFlN9IRrmnr92LmJ9wfCWKmYstXBKzq2YGQi48nC'
ACCESS_TOKEN = '1565097836-MEdliZfmoXqUcBYGY4ibLz3APLycoJRNaPFeJR7'
ACCESS_TOKEN_SECRET = 'Q6O1CjyV8k11sAu7tPs1iUFnYwdsh21kujiGq8mAhtorW '



In [2]:
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET)
help(twitter)

Help on Twython in module twython.api object:

class Twython(twython.endpoints.EndpointsMixin, builtins.object)
 |  Twython(app_key=None, app_secret=None, oauth_token=None, oauth_token_secret=None, access_token=None, token_type='bearer', oauth_version=1, api_version='1.1', client_args=None, auth_endpoint='authenticate')
 |  
 |  Method resolution order:
 |      Twython
 |      twython.endpoints.EndpointsMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, app_key=None, app_secret=None, oauth_token=None, oauth_token_secret=None, access_token=None, token_type='bearer', oauth_version=1, api_version='1.1', client_args=None, auth_endpoint='authenticate')
 |      Instantiates an instance of Twython. Takes optional parameters for
 |      authentication and such (see below).
 |      
 |      :param app_key: (optional) Your applications key
 |      :param app_secret: (optional) Your applications secret key
 |      :param oauth_token: (optional) When using **OAuth

In [3]:
for status in twitter.search(q='"data science"')["statuses"]:
    user = status["user"]["screen_name"]
    text = status["text"]
    print(user, ":", text)
    print()

IoT_Technojeder : RT @Ronald_vanLoon: The Future of #DataScience in One Picture
 by @granvilledsc @DataScienceCtrl @AnalyticBridge @BoozAllen |
 
 Read more…

TazPoltorak : @kareem_carr It was when I stopped believing that one must be talented and started practicing that I was able to le… https://t.co/DFnYXQCaFN

RobertSun24 : RT @basdjacksilva: “Preparing young people to succeed in the innovation economy with data science, probability and financial literacy has t…

Ronald_vanLoon : The Future of #DataScience in One Picture
 by @granvilledsc @DataScienceCtrl @AnalyticBridge @BoozAllen |
 
 Read m… https://t.co/IWrsfmes4l

eelrekab : RT @eelrekab: Python eats away at R: Top Software for #analytics, #datascience, #machinelearning in 2018 https://t.co/Hwfu4TXT4D

curated_data : Data Science Symposium 2019, Oct 10-11, Cincinnati #kdnuggets #datascientist #symposium #data https://t.co/MCPOEZtUlG

JustBeMentalist : RT @CyberStudents: SAP needs your data science expertise! SAP plans on awardi

In [17]:
"""
This isn't that interesting, largely because the Twitter Search API just shows
    you whatever hardful of recent results it feels like. When you're doint
    data science, more often you want a lot of tweets. This is where
    #STREAMING API is useful.
    
    It allows you to connect to (a sample of) the great Twitter firehose.
    To use it, you'll need to authenticate using your access tokens.
    
    In order to access the Streaming API with Twython, we need to define a class
    that inherits from TwythonStreamer and that overrides its on_success
    method (and possibly its on_error method):
"""

from twython import TwythonStreamer

# appending data to a global variable is pretty poor form
# but it makes the example much simpler
tweets = []

In [18]:
class MyStreamer(TwythonStreamer):
    """our own subclass of TwythonStreamer that specifies how to interact
    with the stream"""
    
    def on_success(self, data):
        """what do we do when twitter sends us data?
        here data will be a Python dict representing a tweet"""
        
        # only want to collect English-language tweets
        if data['lang'] == 'en':
            tweets.append(data)
            print("received tweet #", len(tweets))
            
        # stop when we've collected enough
        if len(tweets >= 1000):
            self.disconnect()
            
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
# MyStreamer will connect to the Twitter stream and wait for Twitte to feed it
#    data. Each time it receives some data (here, a Tweet represented as a
#    Python object) it passes it to the on_success method, which appends it to
#    our tweets list if its language is English, and then disconnects the
#    streamer after it's collected 1000 tweets

In [7]:
# All that's left is to initialize it and start it running:
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
                   ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [13]:
# starts consuming public statuses that contain the keyword 'data'
stream.statuses.filter(track='data')

401 b'<html>\\n<head>\\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\\n<title>Error 401 Unauthorized</title>\n</head>\n<body>\n<h2>HTTP ERROR: 401</h2>\n<p>Problem accessing \'/1.1/statuses/filter.json\'. Reason:\n<pre>    Unauthorized</pre>\n</body>\n</html>\n'


In [9]:
# if instead we wanted to start consuming a sample of "all" public statuses
stream.statuses.sample()

401 b'<html>\\n<head>\\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\\n<title>Error 401 Unauthorized</title>\n</head>\n<body>\n<h2>HTTP ERROR: 401</h2>\n<p>Problem accessing \'/1.1/statuses/sample.json\'. Reason:\n<pre>    Unauthorized</pre>\n</body>\n</html>\n'


In [15]:
# This will run until it collects 1000 tweet(or until it encounters an error)
#    and stop, at which point you can start analyzing those tweets. For instance
#   you could find the most common hashtags with:
from collections import Counter
top_hashtags = Counter(hashtag['text'].lower()
                      for tweet in tweets
                      for hashtag in tweet["entities"]["hashtags"])

print(top_hashtags.most_common(5))

[]


In [19]:
twitter_api_documentation = 'https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object'

In [20]:
# In a non-toy project you probably wouldn’t want to rely on an in-memory list for storing the tweets.
# Instead you’d want to save them to a file or a database, so that you’d have them permanently.

In [21]:
#page 121